In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

In [2]:
from src.datasets.smnistp import SplitMNISTPreprocessed

benchmark = SplitMNISTPreprocessed(
    # n_experiences=5,
    n_experiences=10,
    shuffle=False,
    replace_existing=True,
)

100%|██████████| 40/40 [00:03<00:00, 11.49it/s]


In [3]:
from src.ddvfa import DDVFAStrategy

# Create the Strategy Instance
cl_strategy = DDVFAStrategy(
    projectdir(),   
)

# Training Loop
print('Starting experiment...')

results = []
train_results = []
for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    train_results.append(cl_strategy.train(experience))
    print('Training completed')

    print('Computing accuracy on the current test set')
    results.append(cl_strategy.eval(benchmark.test_stream[exp_id]))
    # results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience  0
5923


 67%|██████▋   | 16/24 [00:52<00:26,  3.28s/it, jl time=4.68]


KeyboardInterrupt: 

In [ ]:
benchmark.train_stream.__getitem__(0).dataset[0][0]

tensor([0.5175, 0.5121, 0.5146,  ..., 0.4809, 0.4826, 0.5080])

In [ ]:
print(results, train_results)

[1.0, 0.4152027027027027] [1.0, 1.0]


In [ ]:
benchmark.train_stream[0].classes_in_this_experience

[0]

In [ ]:
import numpy as np

jl.x = np.random.rand(49)
jl.y = 0
jl.eval("train!(art, x, y=y)")


1

In [ ]:
from julia import Main as jl

print(jl.eval("AdaptiveResonance.get_n_weights(art)"))
print(jl.eval("size(art.F2)"))
print(jl.eval("art.labels"))
print(1 == jl.eval("length(unique(art.labels))"))
print(type(jl.eval("unique(art.labels)")))

1
(1,)
[1]
True
<class 'numpy.ndarray'>
